In [1]:
from tabulate import tabulate
from termcolor import colored
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='chinna',
    database='lms',
)
cursor = connection.cursor()

def Issue_book():
    print("Enter book issuance details:")
    try:
        member_id = int(input("Enter the member ID: "))
        book_id = int(input("Enter the book ID: "))
        
        # Check if the member exists
        sel_member = "SELECT * FROM Member WHERE Member_id = %s"
        cursor.execute(sel_member, (member_id,))
        member_record = cursor.fetchone()
        
        # Check if the book exists and has available copies
        sel_book = "SELECT * FROM Book WHERE Book_id = %s AND Copies > 0"
        cursor.execute(sel_book, (book_id,))
        book_record = cursor.fetchone()
        
        if member_record and book_record:
            issue_id = int(input("Enter an issue ID: "))
            issue_date = input("Enter a date in YYYY-MM-DD format: ")
            copies1 = int(input("Enter the number of copies to issue: "))
            
            if copies1 > 0 and copies1 <= book_record[4]:  # Assuming Copies is at index 4
                # Insert into issue table
                ins_issue = 'INSERT INTO Issue1(Issue_id, Issue_date, Member_id, Book_id, Copies1) VALUES (%s, %s, %s, %s, %s)'
                ins_values_issue = (issue_id, issue_date, member_id, book_id, copies1)
                cursor.execute(ins_issue, ins_values_issue)
                
                # Update Rem_copies in the book table
                update_copies = "UPDATE Book SET Rem_copies = Copies - %s WHERE Book_id = %s"
                cursor.execute(update_copies, (copies1, book_id))
                
                connection.commit()
                print("Book issuance successful!")
            else:
                print("Invalid number of copies or insufficient available copies.")
        else:
            print('No such member exists or no available copies of the book')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")

def tracking_book():
    print("Search book details")
    try:
        book_id = input("Enter book ID: ")
        query = """
                SELECT
                    i.Issue_id,
                    i.Issue_date,
                    m.M_name AS member_name,
                    b.BName AS Book_name,
                    i.Copies1
                FROM
                    Issue1 i
                JOIN
                    Member m ON i.member_id = m.member_id
                JOIN
                    Book b ON i.Book_id = b.Book_id
                WHERE
                    i.book_id = %s;
                """

        cursor.execute(query, (book_id,))
        results = cursor.fetchall()

        if results:
            headers = [i[0] for i in cursor.description]
            colored_headers = [colored(header, 'yellow') for header in headers]
            print(tabulate(results, headers=colored_headers, tablefmt="pretty"))
            print('Book Search successfully')
        else:
            print("No books found for book ID:", book_id)

    except mysql.connector.Error as error:
        print("MySQL error:", error)



def Return_book():
    print("Enter book return details:")
    try:
        issue_id = int(input("Enter the issue ID: "))
        
        sel_issue = "SELECT * FROM issue1 WHERE issue_id = %s"
        cursor.execute(sel_issue, (issue_id,))
        issue_record = cursor.fetchone()
        
        if issue_record:
            print("Issue Record:", issue_record)

            # Accessing specific indices
            copies = issue_record[4]  # Assuming copies are at index 4
            book_id = issue_record[3]  # Assuming book_id is at index 3
            print("Copies Returned:", copies)
            print("Book ID:", book_id)

            return_id = int(input("Enter a return ID: "))
            return_date = input("Enter a date in YYYY-MM-DD format: ")
            member_id = int(input("Enter the member ID: "))
            book_id = int(input("Enter the book ID: "))
            
            # Insert return details into the 'returns' table
            ins_return = 'INSERT INTO return3 (return_id, return_date, member_id, book_id, copies) VALUES (%s, %s, %s, %s, %s)'
            ins_values_return = (return_id, return_date, member_id, book_id, copies)
            cursor.execute(ins_return, ins_values_return)
            
            # Delete the issue record from the 'issue1' table
            del_issue = "DELETE FROM issue1 WHERE issue_id = %s"
            cursor.execute(del_issue, (issue_id,))
            
            # Update 'rem_copie' in the 'book' table
            update_copies = "UPDATE book SET Rem_copies = Rem_copies + %s WHERE book_id = %s"
            cursor.execute(update_copies, (copies, book_id))
            
            connection.commit()
            print("Book return successful!")
        else:
            print('No such issued book exists')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")
def track_return():
    print("Search return details")
    try:
        book_id = input("Enter book ID: ")
        query = """
        SELECT r.*, M_name AS member_name, b.bname AS book_name
        FROM return3 r
        JOIN member m ON r.member_id = m.member_id
        JOIN book b ON r.book_id = b.book_id
        WHERE r.book_id = %s
        """
        cursor.execute(query, (book_id,))
        results = cursor.fetchall()
        if results:
            headers = [i[0] for i in cursor.description]
            colored_headers = [colored(header, 'yellow') for header in headers]
            print(tabulate(results, headers=colored_headers, tablefmt="pretty"))
            print('Book Search successful')
        else:
            print("No returns found for book ID:", book_id)

    except mysql.connector.Error as error:
        print("MySQL error:", error)

print("ok")        

ok
